In [2]:
import numpy as np
import tensorflow as tf
import pickle

In [3]:
pickle_file = 'notMNIST.pickle'
with open(pickle_file,'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [21]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [6]:
# Logistic regression with l2 loss function
# multinomial logistic regression 
train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    
    #Input Data
    tf_train_dataset = tf.constant(train_dataset[:train_subset,:])
    tf_train_labels = tf.constant(train_labels[:train_subset,:])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables
    weights = tf.Variable(
    tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Training computation
    logits = tf.matmul(tf_train_dataset,weights)+biases
    
    #loss funciton using l2 reguralization
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_train_labels))
    loss = tf.reduce_mean(loss + beta*tf.nn.l2_loss(weights))
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Predictions for the training, validation, test data set
    train_prediciton = tf.nn.softmax(logits)
    valid_prediction =  tf.nn.softmax(tf.matmul(tf_valid_dataset,weights)+ biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [13]:
num_steps = 801
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        _,l,prediction = sess.run([optimizer,loss,train_prediciton])
        
        if step % 100 == 0:
            print("Loss at step %d: %f" % (step,l))
            print("Training accuracyL %f%%"%accuracy(prediction,train_labels[:train_subset, :]))
            print("Validation accuracy: %.1f%%"%accuracy(valid_prediction.eval(),valid_labels))
            print("Test accuracy: %.1f%%"%accuracy(test_prediction.eval(),test_labels))

Initialized
Loss at step 0: 46.009590
Training accuracyL 12.110000%
Validation accuracy: 16.7%
Test accuracy: 17.5%
Loss at step 100: 11.852741
Training accuracyL 73.760000%
Validation accuracy: 73.7%
Test accuracy: 80.3%
Loss at step 200: 4.483960
Training accuracyL 78.750000%
Validation accuracy: 78.0%
Test accuracy: 84.6%
Loss at step 300: 1.982266
Training accuracyL 81.890000%
Validation accuracy: 80.6%
Test accuracy: 87.2%
Loss at step 400: 1.130756
Training accuracyL 83.680000%
Validation accuracy: 82.2%
Test accuracy: 88.3%
Loss at step 500: 0.837687
Training accuracyL 84.380000%
Validation accuracy: 82.8%
Test accuracy: 88.5%
Loss at step 600: 0.735707
Training accuracyL 84.630000%
Validation accuracy: 82.9%
Test accuracy: 88.5%
Loss at step 700: 0.699915
Training accuracyL 84.650000%
Validation accuracy: 82.9%
Test accuracy: 88.5%
Loss at step 800: 0.687265
Training accuracyL 84.650000%
Validation accuracy: 83.0%
Test accuracy: 88.6%


In [27]:
#Neural Network Model
batch_size =128
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    #Input data
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,image_size*image_size)) #Placeholder dataset
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels)) #Placeholder Labels
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variables
    weights = tf.Variable(tf.truncated_normal([image_size*image_size,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    #Trianing Computation
    logits = tf.matmul(tf_train_dataset,weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels,logits = logits))
    loss = tf.reduce_mean(loss + beta*tf.nn.l2_loss(weights))
    
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Prediction of training, valid and test data
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_dataset,weights) +biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_dataset,weights) + biases)

In [34]:
num_steps = 3001

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step*batch_size)%(train_labels.shape[0]-batch_size)
        
        batch_data = train_dataset[offset:offset+batch_size,:]
        batch_labels = train_labels[offset:offset + batch_size,:]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        
        _,l,prediction = sess.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        if(step %500 == 0):
            print("Minibatch loss at step %d: %f"%(step,l))
            print('Minibatch accuracy: %.1f%%'%accuracy(prediction,batch_labels))
            print("Validation accuracy : %.1f%%"%accuracy(valid_prediction.eval(),valid_labels))
            print("Test accuracy: %.1f%%"%accuracy(test_prediction.eval(),test_labels))

Initialized
Minibatch loss at step 0: 50.664722
Minibatch accuracy: 7.0%
Validation accuracy : 9.0%
Test accuracy: 9.0%
Minibatch loss at step 500: 1.114048
Minibatch accuracy: 74.2%
Validation accuracy : 80.1%
Test accuracy: 85.9%
Minibatch loss at step 1000: 0.972003
Minibatch accuracy: 75.8%
Validation accuracy : 82.3%
Test accuracy: 87.9%
Minibatch loss at step 1500: 0.753501
Minibatch accuracy: 85.2%
Validation accuracy : 81.4%
Test accuracy: 87.1%
Minibatch loss at step 2000: 0.843287
Minibatch accuracy: 78.1%
Validation accuracy : 80.8%
Test accuracy: 86.3%
Minibatch loss at step 2500: 0.982983
Minibatch accuracy: 74.2%
Validation accuracy : 82.0%
Test accuracy: 87.9%
Minibatch loss at step 3000: 0.689573
Minibatch accuracy: 82.8%
Validation accuracy : 81.5%
Test accuracy: 87.0%


In [35]:
#Logistic Regression with l2 loss and small traning data
# multinomial logistic regression 
train_subset = 100
beta = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables    
    weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # loss function using l2
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean(loss + beta * tf.nn.l2_loss(weights) )
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases )
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [37]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        
        _, l, predictions = session.run([optimizer, loss, train_prediction])

        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
            print('Validation accuracy: %.1f%%' % accuracy( valid_prediction.eval(), valid_labels) )

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 45.902740
Training accuracy: 6.0%
Validation accuracy: 14.4%
Loss at step 100: 11.073853
Training accuracy: 100.0%
Validation accuracy: 48.9%
Loss at step 200: 4.131681
Training accuracy: 100.0%
Validation accuracy: 54.8%
Loss at step 300: 1.593157
Training accuracy: 100.0%
Validation accuracy: 60.3%
Loss at step 400: 0.666852
Training accuracy: 100.0%
Validation accuracy: 63.9%
Loss at step 500: 0.328921
Training accuracy: 100.0%
Validation accuracy: 65.9%
Loss at step 600: 0.205433
Training accuracy: 100.0%
Validation accuracy: 66.7%
Loss at step 700: 0.160130
Training accuracy: 100.0%
Validation accuracy: 67.0%
Loss at step 800: 0.143376
Training accuracy: 100.0%
Validation accuracy: 67.0%
Test accuracy: 73.0%


In [51]:
batch_size = 128
beta = 0.001

graph = tf.Graph()
with graph.as_default():
    #Input Variables
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size,image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size,num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #hidden layer
    hidden_nodes =1024
    hidden_weights = tf.Variable(tf.truncated_normal([image_size*image_size,hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset,hidden_weights)+ hidden_biases)
    
    #add dropout to hidden layer
    temp_prob = tf.placeholder("float")
    hidden_layer_dropout = tf.nn.dropout(hidden_layer,temp_prob)
    
    #Variables
    weights = tf.Variable(tf.truncated_normal([hidden_nodes,num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(hidden_layer_dropout, weights) + biases
    loss = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels) )
    loss = tf.reduce_mean( loss + beta * tf.nn.l2_loss(weights) )

    #Optimizer
    #global_step = tf.Variable(0)  # count the number of steps taken.
    #learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.95, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #Prediction of training , validation and test data
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset,hidden_weights)+hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu, weights)+ biases)
    
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset,hidden_weights)+hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu, weights)+ biases)

In [54]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, temp_prob : 0.5}
        _, l, predictions = session.run( [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy( valid_prediction.eval(), valid_labels) )
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 502.703400
Minibatch accuracy: 6.2%
Validation accuracy: 30.0%
Minibatch loss at step 500: 35.359711
Minibatch accuracy: 64.8%
Validation accuracy: 78.3%
Minibatch loss at step 1000: 15.539696
Minibatch accuracy: 65.6%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 15.145881
Minibatch accuracy: 75.8%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 26.630325
Minibatch accuracy: 65.6%
Validation accuracy: 75.9%
Minibatch loss at step 2500: 12.055209
Minibatch accuracy: 66.4%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 5.227393
Minibatch accuracy: 71.9%
Validation accuracy: 80.8%
Test accuracy: 86.6%
